In [37]:
import keras

import numpy as np
import pandas as pd
from keras.preprocessing import sequence
from keras.models import Sequential
from keras.layers import Dense, Embedding, LSTM,  Dropout
from tensorflow.python.keras.preprocessing.text import Tokenizer
from tensorflow.python.keras.preprocessing.sequence import pad_sequences

from sklearn.feature_extraction.text import CountVectorizer
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense, Embedding, LSTM, SpatialDropout1D
from sklearn.model_selection import train_test_split
from keras.utils.np_utils import to_categorical
import re

import csv
from csv import writer
from csv import DictWriter

import glob

import nltk, re, string, collections
from nltk import word_tokenize 
from nltk.corpus import stopwords
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /usr/share/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [38]:
sentiment_corpus = []

In [39]:
product_comments = pd.read_csv("../input/productcomments/data.csv", encoding = "utf-8", error_bad_lines=False)

for y,d in zip(product_comments['Yorum'],product_comments['Duygu']):
    y = [i.lower() for i in y.split() if re.findall("^[a-zA-Z0-9ğüşöçİĞÜŞÖÇ]+$", i)]
    sentiment_corpus.append([y, d])

In [40]:
path = "../input/sa-imdb/SA_IMDB_translated_TR/TEST/negTr/*.txt"
all_files = glob.glob(path)
for file in all_files:
    lines = open(file).readlines()
    for l in lines:
        l = [i.lower() for i in l.split() if re.findall("^[a-zA-Z0-9ğüşöçİĞÜŞÖÇ]+$", i)]
        sentiment_corpus.append([l, 0])
        
path = "../input/sa-imdb/SA_IMDB_translated_TR/TEST/posTr/*.txt"
all_files = glob.glob(path)
for file in all_files:
    lines = open(file).readlines()
    for l in lines:
        l = [i.lower() for i in l.split() if re.findall("^[a-zA-Z0-9ğüşöçİĞÜŞÖÇ]+$", i)]
        sentiment_corpus.append([l, 1])

path = "../input/sa-imdb/SA_IMDB_translated_TR/TRAIN/negTr/*.txt"
all_files = glob.glob(path)
for file in all_files:
    lines = open(file).readlines()
    for l in lines:
        l = [i.lower() for i in l.split() if re.findall("^[a-zA-Z0-9ğüşöçİĞÜŞÖÇ]+$", i)]
        sentiment_corpus.append([l, 0])

path = "../input/sa-imdb/SA_IMDB_translated_TR/TRAIN/posTr/*.txt"
all_files = glob.glob(path)
for file in all_files:
    lines = open(file).readlines()
    for l in lines:
        l = [i.lower() for i in l.split() if re.findall("^[a-zA-Z0-9ğüşöçİĞÜŞÖÇ]+$", i)]
        sentiment_corpus.append([l, 1])


KeyboardInterrupt: 

In [ ]:
tr_reviews = open("../input/sa-tr-reviews/reviews.neg").readlines()
for x in tr_reviews:
    x = [i.lower() for i in x.split() if re.findall("^[a-zA-Z0-9ğüşöçİĞÜŞÖÇ]+$", i)]
    sentiment_corpus.append([x, 0])

tr_reviews = open("../input/sa-tr-reviews/reviews.pos").readlines()
for x in tr_reviews:
    x = [i.lower() for i in x.split() if re.findall("^[a-zA-Z0-9ğüşöçİĞÜŞÖÇ]+$", i)]
    sentiment_corpus.append([x, 1])

In [ ]:
# for i in sentiment_corpus:
#     print(i[0],i[1])

In [ ]:
my_array =np.array(sentiment_corpus)
df = pd.DataFrame(my_array, columns = ['text','sentiment'])

In [ ]:
df

In [ ]:
print(df[df['sentiment'] == 1].size)
print(df[df['sentiment'] == 0].size)
    
max_fatures = 236141
tokenizer = Tokenizer(num_words=max_fatures, split=' ')
tokenizer.fit_on_texts(df['text'].values)
X = tokenizer.texts_to_sequences(df['text'].values)
X = pad_sequences(X)

In [ ]:
embed_dim = 128
lstm_out = 196

model = Sequential()
model.add(Embedding(max_fatures, embed_dim,input_length = X.shape[1]))
model.add(SpatialDropout1D(0.4))
model.add(LSTM(lstm_out, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(2,activation='softmax'))
model.compile(loss = 'categorical_crossentropy', optimizer='adam',metrics = ['accuracy'])
print(model.summary())

In [ ]:
Y = pd.get_dummies(df['sentiment']).values
X_train, X_test, Y_train, Y_test = train_test_split(X,Y, test_size = 0.33, random_state = 42)
print(X_train.shape,Y_train.shape)
print(X_test.shape,Y_test.shape)

In [ ]:
model.fit(X_train, Y_train, epochs = 7, batch_size=256, verbose = 1)

In [ ]:
validation_size = 1500

X_validate = X_test[-validation_size:]
Y_validate = Y_test[-validation_size:]
X_test = X_test[:-validation_size]
Y_test = Y_test[:-validation_size]
score,acc = model.evaluate(X_test, Y_test, verbose = 1, batch_size = 256)
print("score: %.2f" % (score))
print("acc: %.2f" % (acc))

In [ ]:
pos_cnt, neg_cnt, pos_correct, neg_correct = 0, 0, 0, 0
for x in range(len(X_validate)):
    
    result = model.predict(X_validate[x].reshape(1,X_test.shape[1]),batch_size=256,verbose = 1)[0]
   
    if np.argmax(result) == np.argmax(Y_validate[x]):
        if np.argmax(Y_validate[x]) == 0:
            neg_correct += 1
        else:
            pos_correct += 1
       
    if np.argmax(Y_validate[x]) == 0:
        neg_cnt += 1
    else:
        pos_cnt += 1


print("pos_acc", pos_correct/pos_cnt*100, "%")
print("neg_acc", neg_correct/neg_cnt*100, "%")

In [ ]:
print("../input/tbmmcorpusdonem2027/NLP/donem25/yıl2/1.txt")

demo = ["Domates üreticilerinin ürünlerinin satışında yaşadıkları sorunlara, İstanbul Milletvekili Emine Beyza Üstün'ün cevabı."]
print(demo)
demo = tokenizer.texts_to_sequences(demo)
demo = pad_sequences(demo, maxlen=28, dtype='int32', value=0)
print(demo)
sentiment = model.predict(demo,batch_size=256,verbose = 2)[0]
if(np.argmax(sentiment) == 0):
    print("negative")
elif (np.argmax(sentiment) == 1):
    print("positive")